In [2]:
import re
rx = r'^(\w) (\d+) \(#(.*)\)$'

def direction(d):
  if d == 'R':
    return (1, 0)
  elif d == 'L':
    return (-1, 0)
  elif d == 'U':
    return (0, -1)
  elif d == 'D':
    return (0, 1)

def parse(input):
  lines = [l.strip() for l in open(input, 'r').readlines()]
  groups = [re.match(rx, l).groups() for l in lines]
  return [(direction(a), int(b), c) for a, b, c in groups]

parse('sample')

[((1, 0), 6, '70c710'),
 ((0, 1), 5, '0dc571'),
 ((-1, 0), 2, '5713f0'),
 ((0, 1), 2, 'd2c081'),
 ((1, 0), 2, '59c680'),
 ((0, 1), 2, '411b91'),
 ((-1, 0), 5, '8ceee2'),
 ((0, -1), 2, 'caa173'),
 ((-1, 0), 1, '1b58a2'),
 ((0, -1), 2, 'caa171'),
 ((1, 0), 2, '7807d2'),
 ((0, -1), 3, 'a77fa3'),
 ((-1, 0), 2, '015232'),
 ((0, -1), 2, '7a21e3')]

In [18]:
def add(a, b, f = 1):
  return (a[0] + b[0] * f, a[1] + b[1] * f)

def get_rotation(a, b):
  if a == (1, 0): return 90 if b == (0, 1) else -90
  if a == (-1, 0): return -90 if b == (0, 1) else 90
  if a == (0, 1): return 90 if b == (-1, 0) else -90
  if a == (0, -1): return -90 if b == (-1, 0) else 90

def get_to_the_right(current, next):
  cx, cy = current
  nx, ny = next
  dx = nx - cx
  dy = ny - cy
  yield (cx - dy, cy + dx)
  yield (cx + dx - dy, cy + dx + dy)

def get_to_the_left(current, next):
  cx, cy = current
  nx, ny = next
  dx = nx - cx
  dy = ny - cy
  yield (cx + dy, cy - dx)
  yield (cx + dx + dy, cy + dy - dx)

def neighbours(current):
  cx, cy = current
  yield (cx - 1, cy)
  yield (cx + 1, cy)
  yield (cx, cy - 1)
  yield (cx, cy + 1)

def first_part(input):
  current = (0, 0)
  nw = se = current

  for direction, f, _ in input:
    current = add(current, direction, f)
    nw = (min(nw[0], current[0]), min(nw[1], current[1]))
    se = (max(se[0], current[0]), max(se[1], current[1]))

  size = add(se, nw, -1)
  map = [['.'] * (size[0] + 1) for _ in range(size[1] + 1)]

  current = add((0, 0), nw, -1)
  start_rotation = None
  rotation = 0
  left = []
  right = []
  for direction, f, _ in input:
    if start_rotation is None:
      start_rotation = direction
    else:
      rotation += get_rotation(previous_direction, direction)
    previous_direction = direction
    for _ in range(f):
      np = add(current, direction)
      left += get_to_the_left(current, np)
      right += get_to_the_right(current, np)
      map[current[1]][current[0]] = '#'
      current = np
  
  rotation += get_rotation(previous_direction, start_rotation)
  inside = right if rotation == 360 else left
  
  while inside:
    current = inside.pop()
    if current[0] < 0 or current[1] < 0 or current[0] > size[0] or current[1] > size[1]:
      continue
    if map[current[1]][current[0]] == '#':
      continue
    map[current[1]][current[0]] = '#'
    inside += neighbours(current)
  
  # count '#'
  return sum([sum([1 if c == '#' else 0 for c in row]) for row in map])

assert first_part(parse('sample')) == 62
first_part(parse('input'))

33491

In [102]:
from collections import defaultdict

D = [(1, 0), (0, 1), (-1, 0), (0, -1)]

def intersect_ranges(a, b):
  return (max(a[0], b[0]), min(a[1], b[1]))

def second_part(input):
  input = [(int(c[:5], 16), D[int(c[5])]) for c in [c for _, _, c in input]]

  current = (0, 0)
  lines = []
  ys = set([0])
  for steps, direction in input:
    next = add(current, direction, steps)
    ys.add(next[1])
    line = (current, next)
    if current[0] > next[0] if direction[1] == 0 else current[1] > next[1]:
      line = (next, current)
    lines.append(line)
    current = next

  cross = defaultdict(list)
  for y in sorted(ys):
    for line in lines:
      if line[0][1] <= y <= line[1][1]:
        cross[y].append(line)
  
  lines = sorted(cross.items(), key=lambda x: x[0])
  
  result = 0
  width = 0
  py = None
  vertical = None
  for y, lines in lines:
    lines = sorted(lines, key=lambda x: x[0][0])
    
    if py is not None:
      result += width * (y - py - 1)
    width = 0
    x = None

    next_vertical = []
    for line in lines:
      if line[0][1] == line[1][1]:
        result += line[1][0] - line[0][0] + 1
      elif line[1][1] != y:
        if line[0][1] != y:
          result += 1
        if x is None:
          x = line[0][0]
        else:
          width += line[0][0] - x + 1
          next_vertical.append((x, line[0][0]))
          x = None

    if vertical is not None:
      intersections = []
      for r in next_vertical:
        for v in vertical:
          intersections.append(intersect_ranges(r, v))
      for r in intersections:
        if r[0] < r[1]:
          result += r[1] - r[0] - 1

    vertical = next_vertical
    py = y

  return result

assert second_part(parse('sample')) == 952408144115
second_part(parse('input'))

87716969654406